# Model Context Protocol (MCP) Tutorial with LangChain/LangGraph

## Table of Contents
1. [Introduction to MCP](#introduction)
2. [Core Concepts and Architecture](#core-concepts)
3. [Setting Up Your Environment](#setup)
4. [Basic MCP Examples](#basic-examples)
5. [Integrating with LangChain](#langchain-integration)
6. [Advanced Examples with LangGraph](#langgraph-examples)
7. [Building Custom MCP Servers](#custom-servers)
8. [Best Practices and Patterns](#best-practices)

---

In [ ]:
prompt = f"""You have access to the tracking system at {trackin_url}.
Current user is {user_id} with the following permissions: {permissions}"""

mcp_context= {
    "request": {
        "intent": "track_shipment",
        "shipping_id": "12345"
    },
    "context":{
        "resources":[
            "uri": "...",
            "uri": "..."
        ]
    
    
}

## 1. Introduction to MCP <a id="introduction"></a>

The Model Context Protocol (MCP) is an open protocol that standardizes how AI assistants connect with external data sources and tools. Think of it as a universal adapter that allows AI models to interact with various services in a consistent way.

### Why MCP Matters

Before MCP, integrating AI models with external tools required custom implementations for each service. MCP provides:
- **Standardization**: One protocol for all integrations
- **Interoperability**: Tools built for one AI system work with others
- **Security**: Built-in safety features for tool execution
- **Flexibility**: Support for various data sources and operations

### Key Components

1. **MCP Servers**: Expose resources, tools, and prompts to AI systems
2. **MCP Clients**: Connect to servers and use their capabilities
3. **Resources**: Data sources that can be read by the AI
4. **Tools**: Functions the AI can execute
5. **Prompts**: Pre-configured templates for common tasks

## 2. Core Concepts and Architecture <a id="core-concepts"></a>

### MCP Architecture Overview

```
┌─────────────┐     MCP Protocol     ┌─────────────┐
│   AI Model  │ ◄──────────────────► │ MCP Server  │
│  (Client)   │                      │             │
└─────────────┘                      └─────────────┘
       │                                     │
       │                                     │
       ▼                                     ▼
┌─────────────┐                      ┌─────────────┐
│  LangChain  │                      │  External   │
│  Framework  │                      │  Services   │
└─────────────┘                      └─────────────┘
```

### Protocol Features

MCP uses JSON-RPC 2.0 for communication and supports:
- **Bidirectional communication**: Both client and server can initiate requests
- **Streaming**: For real-time data updates
- **Progress tracking**: Monitor long-running operations
- **Error handling**: Standardized error responses

## 3. Setting Up Your Environment <a id="setup"></a>

Let's start by installing the necessary packages and setting up our environment.

In [6]:
# Install required packages
!pip install mcp langchain langchain-mcp langgraph anthropic python-dotenv

# For MCP server development
!pip install uvicorn fastapi

In [ ]:
# Import necessary libraries
import os
import json
import asyncio
from typing import List, Dict, Any, Optional
from datetime import datetime

# MCP imports
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# LangChain imports
from langchain_core.tools import Tool
from langchain_core.messages import HumanMessage, AIMessage
from langchain_mcp import create_mcp_tools
from langchain.agents import AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate

# LangGraph imports
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode

# Set up environment variables
from dotenv import load_dotenv
load_dotenv()

print("Environment setup complete!")

## 4. Basic MCP Examples <a id="basic-examples"></a>

Let's start with basic examples to understand how MCP works.

### Example 1: Connecting to an MCP Server

First, let's create a simple connection to an MCP server. This example shows the fundamental pattern of establishing a client-server connection.

In [ ]:
async def connect_to_mcp_server(server_command: List[str]):
    """
    Establishes a connection to an MCP server.
    
    Args:
        server_command: Command to start the MCP server (e.g., ['node', 'server.js'])
    
    Returns:
        ClientSession: Active MCP client session
    """
    # Create server parameters
    server_params = StdioServerParameters(
        command=server_command[0],
        args=server_command[1:] if len(server_command) > 1 else [],
        env=None  # Use current environment
    )
    
    # Start the client connection
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()
            
            # Get server information
            info = await session.get_server_info()
            print(f"Connected to: {info.name}")
            print(f"Version: {info.version}")
            
            return session

# Example usage (requires an actual MCP server)
# await connect_to_mcp_server(['python', 'my_mcp_server.py'])

### Example 2: Working with MCP Resources

Resources in MCP are data sources that the AI can read. Let's explore how to list and read resources.

In [ ]:
async def explore_mcp_resources(session: ClientSession):
    """
    Explores available resources from an MCP server.
    
    This function demonstrates:
    1. Listing all available resources
    2. Reading specific resource content
    3. Handling resource metadata
    """
    # List all available resources
    resources = await session.list_resources()
    
    print("Available Resources:")
    print("-" * 50)
    
    for resource in resources:
        print(f"\nResource: {resource.uri}")
        print(f"Name: {resource.name}")
        print(f"Description: {resource.description}")
        print(f"MIME Type: {resource.mimeType}")
        
        # Read the resource content
        try:
            content = await session.read_resource(resource.uri)
            print(f"Content Preview: {content.text[:200]}..." if len(content.text) > 200 else f"Content: {content.text}")
        except Exception as e:
            print(f"Error reading resource: {e}")

# Example of a mock resource for demonstration
class MockResource:
    def __init__(self, uri: str, name: str, description: str, content: str):
        self.uri = uri
        self.name = name
        self.description = description
        self.mimeType = "text/plain"
        self.content = content

# Simulate resource exploration
mock_resources = [
    MockResource(
        "file:///data/customers.json",
        "Customer Database",
        "JSON file containing customer information",
        '{"customers": [{"id": 1, "name": "Alice"}, {"id": 2, "name": "Bob"}]}'
    ),
    MockResource(
        "api://weather/current",
        "Current Weather",
        "Real-time weather data",
        '{"temperature": 72, "conditions": "sunny", "humidity": 45}'
    )
]

print("Simulated Resource Exploration:")
for resource in mock_resources:
    print(f"\nResource: {resource.uri}")
    print(f"Content: {resource.content}")

### Example 3: Using MCP Tools

Tools in MCP are functions that the AI can execute. They're similar to function calling in LLMs but standardized across different systems.

In [ ]:
async def demonstrate_mcp_tools(session: ClientSession):
    """
    Demonstrates how to work with MCP tools.
    
    Tools are executable functions that the AI can call.
    Each tool has:
    - A unique name
    - Input schema (JSON Schema format)
    - Description for the AI to understand its purpose
    """
    # List available tools
    tools = await session.list_tools()
    
    print("Available Tools:")
    print("=" * 60)
    
    for tool in tools:
        print(f"\nTool: {tool.name}")
        print(f"Description: {tool.description}")
        print(f"Input Schema: {json.dumps(tool.inputSchema, indent=2)}")
        
        # Example: Call a tool
        if tool.name == "calculate":
            # Prepare tool arguments based on the schema
            arguments = {
                "operation": "add",
                "a": 10,
                "b": 20
            }
            
            try:
                result = await session.call_tool(tool.name, arguments=arguments)
                print(f"\nTool Result: {result}")
            except Exception as e:
                print(f"Error calling tool: {e}")

# Simulate tool demonstration
print("Simulated MCP Tools:")
print("=" * 60)

example_tools = [
    {
        "name": "search_database",
        "description": "Search customer database with filters",
        "inputSchema": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "Search query"},
                "filters": {
                    "type": "object",
                    "properties": {
                        "status": {"type": "string", "enum": ["active", "inactive"]},
                        "created_after": {"type": "string", "format": "date"}
                    }
                }
            },
            "required": ["query"]
        }
    },
    {
        "name": "send_email",
        "description": "Send an email to a recipient",
        "inputSchema": {
            "type": "object",
            "properties": {
                "to": {"type": "string", "format": "email"},
                "subject": {"type": "string"},
                "body": {"type": "string"}
            },
            "required": ["to", "subject", "body"]
        }
    }
]

for tool in example_tools:
    print(f"\nTool: {tool['name']}")
    print(f"Description: {tool['description']}")
    print(f"Schema: {json.dumps(tool['inputSchema'], indent=2)}")

## 5. Integrating with LangChain <a id="langchain-integration"></a>

Now let's explore how to integrate MCP with LangChain, which allows us to use MCP tools within LangChain agents and chains.

In [ ]:
# First, let's create a mock MCP tool adapter for demonstration
class MCPToolAdapter:
    """
    Adapts MCP tools to work with LangChain.
    This demonstrates the pattern used by langchain-mcp-adapters.
    """
    def __init__(self, mcp_tool_config):
        self.name = mcp_tool_config["name"]
        self.description = mcp_tool_config["description"]
        self.schema = mcp_tool_config["inputSchema"]
    
    def to_langchain_tool(self) -> Tool:
        """
        Converts an MCP tool to a LangChain Tool.
        """
        def tool_func(**kwargs):
            # In a real implementation, this would call the MCP server
            return f"Executed {self.name} with args: {kwargs}"
        
        return Tool(
            name=self.name,
            description=self.description,
            func=tool_func
        )

# Create LangChain tools from MCP tools
mcp_tools_config = [
    {
        "name": "weather_check",
        "description": "Get current weather for a location",
        "inputSchema": {
            "type": "object",
            "properties": {
                "location": {"type": "string", "description": "City name"}
            },
            "required": ["location"]
        }
    },
    {
        "name": "calculate_distance",
        "description": "Calculate distance between two cities",
        "inputSchema": {
            "type": "object",
            "properties": {
                "city1": {"type": "string"},
                "city2": {"type": "string"}
            },
            "required": ["city1", "city2"]
        }
    }
]

# Convert MCP tools to LangChain tools
langchain_tools = []
for config in mcp_tools_config:
    adapter = MCPToolAdapter(config)
    langchain_tools.append(adapter.to_langchain_tool())

print("Created LangChain tools from MCP:")
for tool in langchain_tools:
    print(f"- {tool.name}: {tool.description}")

### Using MCP Tools with LangChain Agents

Now let's create a LangChain agent that can use MCP tools. This demonstrates how MCP enables tool use across different AI frameworks.

In [ ]:
from langchain.agents import AgentType, initialize_agent
from langchain.llms.fake import FakeListLLM

# For demonstration, we'll use a mock LLM
# In practice, you'd use a real model like ChatAnthropic or ChatOpenAI
mock_llm = FakeListLLM(
    responses=[
        "I need to check the weather in New York.",
        "I'll calculate the distance between New York and Los Angeles.",
        "Based on my findings, the weather in New York is sunny and the distance to LA is approximately 2,800 miles."
    ]
)

# Create an agent with MCP tools
def create_mcp_agent(tools: List[Tool], llm):
    """
    Creates a LangChain agent equipped with MCP tools.
    
    This demonstrates the integration pattern where:
    1. MCP tools are converted to LangChain tools
    2. The agent can dynamically decide which tools to use
    3. Tool results are incorporated into the agent's reasoning
    """
    # Create a custom prompt that explains MCP tools
    prompt_template = """
You are an AI assistant with access to Model Context Protocol (MCP) tools.
These tools allow you to interact with external services and data sources.

Available tools:
{tools}

Use these tools to help answer the user's questions accurately.

Question: {input}
{agent_scratchpad}
"""
    
    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["input", "tools", "agent_scratchpad"]
    )
    
    # Initialize the agent
    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True
    )
    
    return agent

# Create and test the agent
mcp_agent = create_mcp_agent(langchain_tools, mock_llm)

# Example usage
print("\nAgent with MCP Tools Example:")
print("=" * 60)
# In a real scenario, you would run:
# response = mcp_agent.run("What's the weather in New York and how far is it from LA?")
# print(response)

### Real-world MCP Integration Pattern

Here's how you would actually integrate MCP with LangChain using the official adapters:

In [ ]:
async def integrate_mcp_with_langchain():
    """
    Demonstrates the actual integration pattern using langchain-mcp-adapters.
    
    This shows:
    1. Connecting to an MCP server
    2. Converting MCP tools to LangChain tools
    3. Using them in a LangChain application
    """
    # This is the pattern you'd use with real MCP servers
    from langchain_mcp import create_mcp_tools
    
    # Connect to MCP server
    server_params = StdioServerParameters(
        command="npx",
        args=["@modelcontextprotocol/server-filesystem", "/path/to/files"]
    )
    
    # In practice, you'd do something like:
    # async with stdio_client(server_params) as (read, write):
    #     async with ClientSession(read, write) as session:
    #         await session.initialize()
    #         
    #         # Create LangChain tools from MCP session
    #         tools = await create_mcp_tools(session)
    #         
    #         # Use tools in your LangChain application
    #         agent = create_mcp_agent(tools, llm)
    #         result = await agent.arun("Search for customer data files")
    
    # For demonstration, let's show the expected structure
    print("MCP + LangChain Integration Pattern:")
    print("1. Connect to MCP server")
    print("2. Initialize session")
    print("3. Convert MCP tools to LangChain tools")
    print("4. Use in agents, chains, or graphs")
    print("5. MCP handles the actual execution")

# Show the integration pattern
await integrate_mcp_with_langchain()

## 6. Advanced Examples with LangGraph <a id="langgraph-examples"></a>

LangGraph allows us to build more complex, stateful applications. Let's see how MCP tools can be integrated into LangGraph workflows.

In [ ]:
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
import operator

# Define the state for our graph
class MCPAgentState(TypedDict):
    """
    State definition for our MCP-powered agent.
    
    This state tracks:
    - Message history
    - Tool calls and results
    - Current task status
    """
    messages: Annotated[Sequence[BaseMessage], operator.add]
    current_task: str
    task_completed: bool
    tool_results: List[Dict[str, Any]]

# Create nodes for our graph
def task_analyzer(state: MCPAgentState) -> MCPAgentState:
    """
    Analyzes the current task and determines which MCP tools to use.
    
    This node demonstrates how to:
    1. Parse user intent
    2. Map intent to available MCP tools
    3. Prepare for tool execution
    """
    task = state["current_task"]
    
    # Analyze task (in practice, you'd use an LLM here)
    analysis = {
        "weather": "weather_check",
        "distance": "calculate_distance",
        "search": "search_database",
        "email": "send_email"
    }
    
    # Determine required tools
    required_tools = []
    for keyword, tool in analysis.items():
        if keyword in task.lower():
            required_tools.append(tool)
    
    # Update state
    state["messages"].append(
        AIMessage(content=f"I'll use these MCP tools: {required_tools}")
    )
    
    return state

def mcp_tool_executor(state: MCPAgentState) -> MCPAgentState:
    """
    Executes MCP tools based on the task analysis.
    
    In a real implementation, this would:
    1. Connect to the MCP server
    2. Call the appropriate tools
    3. Return results to the state
    """
    # Simulate tool execution
    mock_results = [
        {"tool": "weather_check", "result": "New York: 72°F, Sunny"},
        {"tool": "calculate_distance", "result": "NYC to LA: 2,789 miles"}
    ]
    
    state["tool_results"] = mock_results
    state["messages"].append(
        AIMessage(content="MCP tools executed successfully")
    )
    
    return state

def result_synthesizer(state: MCPAgentState) -> MCPAgentState:
    """
    Synthesizes results from MCP tools into a coherent response.
    """
    results = state["tool_results"]
    
    # Create summary
    summary = "Based on MCP tool results:\n"
    for result in results:
        summary += f"- {result['result']}\n"
    
    state["messages"].append(AIMessage(content=summary))
    state["task_completed"] = True
    
    return state

# Build the LangGraph workflow
def create_mcp_langgraph():
    """
    Creates a LangGraph workflow that integrates MCP tools.
    
    This demonstrates:
    1. Stateful processing with MCP
    2. Conditional routing based on tool availability
    3. Error handling and retries
    """
    # Initialize the graph
    workflow = StateGraph(MCPAgentState)
    
    # Add nodes
    workflow.add_node("analyze_task", task_analyzer)
    workflow.add_node("execute_mcp_tools", mcp_tool_executor)
    workflow.add_node("synthesize_results", result_synthesizer)
    
    # Define edges
    workflow.set_entry_point("analyze_task")
    workflow.add_edge("analyze_task", "execute_mcp_tools")
    workflow.add_edge("execute_mcp_tools", "synthesize_results")
    
    # Conditional edge: check if task is completed
    def should_continue(state: MCPAgentState) -> str:
        if state["task_completed"]:
            return END
        return "analyze_task"
    
    workflow.add_conditional_edges(
        "synthesize_results",
        should_continue
    )
    
    # Compile the graph
    app = workflow.compile()
    
    return app

# Create and visualize the workflow
mcp_graph = create_mcp_langgraph()
print("MCP LangGraph workflow created!")

# Example usage
initial_state = {
    "messages": [HumanMessage(content="Check weather in NYC and distance to LA")],
    "current_task": "Check weather in NYC and distance to LA",
    "task_completed": False,
    "tool_results": []
}

# In practice, you would run:
# final_state = mcp_graph.invoke(initial_state)
# print(final_state["messages"][-1].content)

### Complex MCP Workflow Example

Let's create a more sophisticated example that shows how MCP can power a multi-step research assistant.

In [ ]:
from enum import Enum

class ResearchPhase(str, Enum):
    """Phases of our research workflow"""
    PLANNING = "planning"
    GATHERING = "gathering"
    ANALYZING = "analyzing"
    REPORTING = "reporting"
    COMPLETE = "complete"

class ResearchState(TypedDict):
    """Enhanced state for research workflows"""
    messages: Annotated[Sequence[BaseMessage], operator.add]
    research_topic: str
    current_phase: ResearchPhase
    research_plan: List[str]
    gathered_data: Dict[str, Any]
    analysis_results: Dict[str, Any]
    final_report: Optional[str]
    mcp_resources_used: List[str]
    mcp_tools_used: List[str]

def research_planner(state: ResearchState) -> ResearchState:
    """
    Plans research using available MCP resources and tools.
    
    This demonstrates how to:
    1. Query MCP for available resources
    2. Select appropriate tools for the research
    3. Create an execution plan
    """
    topic = state["research_topic"]
    
    # In practice, you'd query the MCP server for available resources
    available_resources = [
        "database://customers",
        "api://market_data",
        "file://reports/2024"
    ]
    
    available_tools = [
        "search_web",
        "analyze_sentiment",
        "generate_summary",
        "create_visualization"
    ]
    
    # Create research plan
    plan = [
        f"1. Search MCP resources for '{topic}'",
        f"2. Use web search tool for recent information",
        f"3. Analyze sentiment of gathered data",
        f"4. Generate comprehensive summary",
        f"5. Create visualizations of key findings"
    ]
    
    state["research_plan"] = plan
    state["mcp_resources_used"] = available_resources[:2]  # Select relevant resources
    state["mcp_tools_used"] = available_tools[:3]  # Select relevant tools
    state["current_phase"] = ResearchPhase.GATHERING
    
    state["messages"].append(
        AIMessage(content=f"Research plan created. Using {len(state['mcp_resources_used'])} MCP resources and {len(state['mcp_tools_used'])} tools.")
    )
    
    return state

def data_gatherer(state: ResearchState) -> ResearchState:
    """
    Gathers data from MCP resources.
    """
    # Simulate gathering data from MCP resources
    gathered_data = {
        "database_results": [
            {"id": 1, "relevance": "high", "data": "Customer feedback on topic"},
            {"id": 2, "relevance": "medium", "data": "Historical trends"}
        ],
        "api_results": {
            "market_trends": "positive",
            "competitor_analysis": "3 main competitors identified"
        },
        "web_search_results": [
            "Recent news article 1",
            "Industry report 2024",
            "Expert opinion piece"
        ]
    }
    
    state["gathered_data"] = gathered_data
    state["current_phase"] = ResearchPhase.ANALYZING
    
    state["messages"].append(
        AIMessage(content="Data gathering complete. Retrieved data from all MCP sources.")
    )
    
    return state

def data_analyzer(state: ResearchState) -> ResearchState:
    """
    Analyzes gathered data using MCP tools.
    """
    # Simulate analysis using MCP tools
    analysis_results = {
        "sentiment_analysis": {
            "overall": "positive",
            "confidence": 0.85,
            "key_themes": ["innovation", "growth", "customer satisfaction"]
        },
        "trend_analysis": {
            "direction": "upward",
            "growth_rate": "15% YoY",
            "forecast": "continued growth expected"
        },
        "key_insights": [
            "Strong market position",
            "Customer satisfaction above industry average",
            "Opportunity for expansion in emerging markets"
        ]
    }
    
    state["analysis_results"] = analysis_results
    state["current_phase"] = ResearchPhase.REPORTING
    
    state["messages"].append(
        AIMessage(content="Analysis complete. Key insights identified.")
    )
    
    return state

def report_generator(state: ResearchState) -> ResearchState:
    """
    Generates final report using MCP tools.
    """
    # Generate comprehensive report
    report = f"""
# Research Report: {state['research_topic']}

## Executive Summary
This research utilized {len(state['mcp_resources_used'])} MCP resources and {len(state['mcp_tools_used'])} MCP tools to analyze the topic comprehensively.

## Key Findings
- Overall sentiment: {state['analysis_results']['sentiment_analysis']['overall']}
- Growth trend: {state['analysis_results']['trend_analysis']['growth_rate']}
- Market position: Strong

## Detailed Analysis
{json.dumps(state['analysis_results'], indent=2)}

## Data Sources
- MCP Resources: {', '.join(state['mcp_resources_used'])}
- MCP Tools: {', '.join(state['mcp_tools_used'])}

## Recommendations
1. Leverage positive market sentiment
2. Expand into identified growth areas
3. Continue monitoring using MCP tools
"""
    
    state["final_report"] = report
    state["current_phase"] = ResearchPhase.COMPLETE
    
    state["messages"].append(
        AIMessage(content="Research complete. Final report generated.")
    )
    
    return state

# Create the research workflow
def create_research_workflow():
    workflow = StateGraph(ResearchState)
    
    # Add all nodes
    workflow.add_node("plan", research_planner)
    workflow.add_node("gather", data_gatherer)
    workflow.add_node("analyze", data_analyzer)
    workflow.add_node("report", report_generator)
    
    # Set up edges
    workflow.set_entry_point("plan")
    workflow.add_edge("plan", "gather")
    workflow.add_edge("gather", "analyze")
    workflow.add_edge("analyze", "report")
    
    # Add conditional routing
    def route_next(state: ResearchState) -> str:
        if state["current_phase"] == ResearchPhase.COMPLETE:
            return END
        return "plan"  # For iterative research
    
    workflow.add_conditional_edges("report", route_next)
    
    return workflow.compile()

# Create and demonstrate the workflow
research_app = create_research_workflow()
print("Advanced MCP Research Workflow created!")
print("\nThis workflow demonstrates:")
print("- Multi-phase processing with MCP")
print("- Resource and tool selection")
print("- Data gathering from multiple MCP sources")
print("- Analysis using MCP tools")
print("- Report generation")

## 7. Building Custom MCP Servers <a id="custom-servers"></a>

Let's explore how to build your own MCP server that can be used by any MCP-compatible client.

In [ ]:
# Example of a custom MCP server implementation
from typing import Optional
from dataclasses import dataclass
import json

@dataclass
class MCPResource:
    """Represents an MCP resource"""
    uri: str
    name: str
    description: str
    mime_type: str = "application/json"

@dataclass
class MCPTool:
    """Represents an MCP tool"""
    name: str
    description: str
    input_schema: Dict[str, Any]

class CustomMCPServer:
    """
    A custom MCP server implementation.
    
    This demonstrates the key components of an MCP server:
    1. Resource management
    2. Tool implementation
    3. Protocol handling
    """
    
    def __init__(self, name: str, version: str):
        self.name = name
        self.version = version
        self.resources: Dict[str, MCPResource] = {}
        self.tools: Dict[str, MCPTool] = {}
        self._setup_default_resources()
        self._setup_default_tools()
    
    def _setup_default_resources(self):
        """Initialize default resources"""
        # Add a sample database resource
        self.resources["db://users"] = MCPResource(
            uri="db://users",
            name="User Database",
            description="Access to user information",
            mime_type="application/json"
        )
        
        # Add a configuration resource
        self.resources["config://settings"] = MCPResource(
            uri="config://settings",
            name="Application Settings",
            description="Current application configuration",
            mime_type="application/json"
        )
    
    def _setup_default_tools(self):
        """Initialize default tools"""
        # User lookup tool
        self.tools["lookup_user"] = MCPTool(
            name="lookup_user",
            description="Look up a user by ID or email",
            input_schema={
                "type": "object",
                "properties": {
                    "identifier": {
                        "type": "string",
                        "description": "User ID or email address"
                    }
                },
                "required": ["identifier"]
            }
        )
        
        # Analytics tool
        self.tools["calculate_metrics"] = MCPTool(
            name="calculate_metrics",
            description="Calculate user engagement metrics",
            input_schema={
                "type": "object",
                "properties": {
                    "metric_type": {
                        "type": "string",
                        "enum": ["engagement", "retention", "growth"],
                        "description": "Type of metric to calculate"
                    },
                    "time_period": {
                        "type": "string",
                        "enum": ["day", "week", "month"],
                        "description": "Time period for calculation"
                    }
                },
                "required": ["metric_type", "time_period"]
            }
        )
    
    async def handle_list_resources(self) -> List[Dict[str, Any]]:
        """Handle list_resources request"""
        return [
            {
                "uri": resource.uri,
                "name": resource.name,
                "description": resource.description,
                "mimeType": resource.mime_type
            }
            for resource in self.resources.values()
        ]
    
    async def handle_read_resource(self, uri: str) -> Dict[str, Any]:
        """Handle read_resource request"""
        if uri not in self.resources:
            raise ValueError(f"Resource not found: {uri}")
        
        # Simulate reading resource data
        if uri == "db://users":
            return {
                "contents": [
                    {
                        "uri": uri,
                        "mimeType": "application/json",
                        "text": json.dumps([
                            {"id": "001", "name": "Alice", "email": "alice@example.com"},
                            {"id": "002", "name": "Bob", "email": "bob@example.com"}
                        ])
                    }
                ]
            }
        elif uri == "config://settings":
            return {
                "contents": [
                    {
                        "uri": uri,
                        "mimeType": "application/json",
                        "text": json.dumps({
                            "theme": "dark",
                            "language": "en",
                            "features": {"analytics": True, "export": True}
                        })
                    }
                ]
            }
    
    async def handle_list_tools(self) -> List[Dict[str, Any]]:
        """Handle list_tools request"""
        return [
            {
                "name": tool.name,
                "description": tool.description,
                "inputSchema": tool.input_schema
            }
            for tool in self.tools.values()
        ]
    
    async def handle_call_tool(self, name: str, arguments: Dict[str, Any]) -> List[Dict[str, Any]]:
        """Handle call_tool request"""
        if name not in self.tools:
            raise ValueError(f"Tool not found: {name}")
        
        # Implement tool logic
        if name == "lookup_user":
            identifier = arguments["identifier"]
            # Simulate user lookup
            result = {
                "id": "001",
                "name": "Alice",
                "email": "alice@example.com",
                "last_login": "2024-01-15T10:30:00Z"
            }
            return [
                {
                    "type": "text",
                    "text": json.dumps(result, indent=2)
                }
            ]
        
        elif name == "calculate_metrics":
            metric_type = arguments["metric_type"]
            time_period = arguments["time_period"]
            # Simulate metric calculation
            result = {
                "metric": metric_type,
                "period": time_period,
                "value": 85.7,
                "change": "+12.3%",
                "trend": "increasing"
            }
            return [
                {
                    "type": "text",
                    "text": json.dumps(result, indent=2)
                }
            ]

# Demonstrate the custom server
custom_server = CustomMCPServer("Custom Analytics Server", "1.0.0")

print("Custom MCP Server Created!")
print(f"Name: {custom_server.name}")
print(f"Version: {custom_server.version}")
print(f"\nResources: {len(custom_server.resources)}")
for uri, resource in custom_server.resources.items():
    print(f"  - {uri}: {resource.name}")
print(f"\nTools: {len(custom_server.tools)}")
for name, tool in custom_server.tools.items():
    print(f"  - {name}: {tool.description}")

### Complete MCP Server with Protocol Implementation

Here's a more complete example showing how to implement the MCP protocol:

In [ ]:
import asyncio
import sys
from typing import AsyncGenerator

class MCPProtocolServer:
    """
    Full MCP protocol implementation for a custom server.
    
    This shows:
    1. JSON-RPC 2.0 message handling
    2. Request/response patterns
    3. Error handling
    4. Streaming support
    """
    
    def __init__(self, server: CustomMCPServer):
        self.server = server
        self.request_handlers = {
            "initialize": self.handle_initialize,
            "resources/list": self.handle_list_resources,
            "resources/read": self.handle_read_resource,
            "tools/list": self.handle_list_tools,
            "tools/call": self.handle_call_tool,
            "prompts/list": self.handle_list_prompts,
            "completion/complete": self.handle_completion
        }
    
    async def handle_initialize(self, params: Dict[str, Any]) -> Dict[str, Any]:
        """Handle initialization request"""
        return {
            "protocolVersion": "2024-10-07",
            "capabilities": {
                "resources": {},
                "tools": {},
                "prompts": {}
            },
            "serverInfo": {
                "name": self.server.name,
                "version": self.server.version
            }
        }
    
    async def handle_list_resources(self, params: Dict[str, Any]) -> Dict[str, Any]:
        """Handle list resources request"""
        resources = await self.server.handle_list_resources()
        return {"resources": resources}
    
    async def handle_read_resource(self, params: Dict[str, Any]) -> Dict[str, Any]:
        """Handle read resource request"""
        uri = params.get("uri")
        if not uri:
            raise ValueError("Missing required parameter: uri")
        
        return await self.server.handle_read_resource(uri)
    
    async def handle_list_tools(self, params: Dict[str, Any]) -> Dict[str, Any]:
        """Handle list tools request"""
        tools = await self.server.handle_list_tools()
        return {"tools": tools}
    
    async def handle_call_tool(self, params: Dict[str, Any]) -> Dict[str, Any]:
        """Handle tool call request"""
        name = params.get("name")
        arguments = params.get("arguments", {})
        
        if not name:
            raise ValueError("Missing required parameter: name")
        
        content = await self.server.handle_call_tool(name, arguments)
        return {"content": content}
    
    async def handle_list_prompts(self, params: Dict[str, Any]) -> Dict[str, Any]:
        """Handle list prompts request"""
        # Example prompts
        prompts = [
            {
                "name": "analyze_user",
                "description": "Analyze user behavior and provide insights",
                "arguments": [
                    {
                        "name": "user_id",
                        "description": "ID of the user to analyze",
                        "required": True
                    }
                ]
            }
        ]
        return {"prompts": prompts}
    
    async def handle_completion(self, params: Dict[str, Any]) -> AsyncGenerator[Dict[str, Any], None]:
        """Handle completion request with streaming"""
        # This demonstrates streaming responses
        prompt = params.get("messages", [])
        
        # Simulate streaming completion
        response_chunks = [
            "Based on the analysis, ",
            "the user shows high engagement ",
            "with an average session duration of 15 minutes."
        ]
        
        for chunk in response_chunks:
            yield {
                "type": "content",
                "content": {"type": "text", "text": chunk}
            }
            await asyncio.sleep(0.1)  # Simulate processing time
    
    async def process_request(self, request: Dict[str, Any]) -> Dict[str, Any]:
        """Process a JSON-RPC request"""
        method = request.get("method")
        params = request.get("params", {})
        request_id = request.get("id")
        
        try:
            if method not in self.request_handlers:
                raise ValueError(f"Unknown method: {method}")
            
            handler = self.request_handlers[method]
            result = await handler(params)
            
            return {
                "jsonrpc": "2.0",
                "id": request_id,
                "result": result
            }
        
        except Exception as e:
            return {
                "jsonrpc": "2.0",
                "id": request_id,
                "error": {
                    "code": -32603,
                    "message": str(e)
                }
            }

# Example of how to run the server
print("MCP Protocol Server Implementation")
print("==================================\n")
print("This server implementation provides:")
print("1. Full JSON-RPC 2.0 support")
print("2. Resource management")
print("3. Tool execution")
print("4. Prompt templates")
print("5. Streaming completions")
print("\nTo run this server:")
print("1. Save as 'my_mcp_server.py'")
print("2. Run with: python my_mcp_server.py")
print("3. Connect from Claude, LangChain, or any MCP client")

## 8. Best Practices and Patterns <a id="best-practices"></a>

Let's explore best practices for working with MCP in production environments.

In [ ]:
# Best Practices Implementation Examples

class MCPBestPractices:
    """
    Demonstrates best practices for MCP implementation.
    """
    
    @staticmethod
    async def secure_tool_execution(tool_name: str, arguments: Dict[str, Any]):
        """
        Best Practice 1: Input Validation and Sanitization
        
        Always validate and sanitize inputs before executing tools.
        """
        # Validate tool name
        allowed_tools = ["search", "calculate", "analyze"]
        if tool_name not in allowed_tools:
            raise ValueError(f"Unauthorized tool: {tool_name}")
        
        # Sanitize arguments
        sanitized_args = {}
        for key, value in arguments.items():
            if isinstance(value, str):
                # Remove potential injection attacks
                sanitized_value = value.replace(";", "").replace("--", "")
                sanitized_args[key] = sanitized_value[:1000]  # Limit length
            else:
                sanitized_args[key] = value
        
        # Execute with timeout
        try:
            result = await asyncio.wait_for(
                execute_tool(tool_name, sanitized_args),
                timeout=30.0  # 30 second timeout
            )
            return result
        except asyncio.TimeoutError:
            raise TimeoutError(f"Tool {tool_name} execution timed out")
    
    @staticmethod
    def implement_rate_limiting():
        """
        Best Practice 2: Rate Limiting
        
        Implement rate limiting to prevent abuse.
        """
        from collections import defaultdict
        from time import time
        
        class RateLimiter:
            def __init__(self, max_requests: int, window_seconds: int):
                self.max_requests = max_requests
                self.window_seconds = window_seconds
                self.requests = defaultdict(list)
            
            def is_allowed(self, client_id: str) -> bool:
                now = time()
                # Clean old requests
                self.requests[client_id] = [
                    req_time for req_time in self.requests[client_id]
                    if now - req_time < self.window_seconds
                ]
                
                # Check rate limit
                if len(self.requests[client_id]) >= self.max_requests:
                    return False
                
                # Record request
                self.requests[client_id].append(now)
                return True
        
        return RateLimiter(max_requests=100, window_seconds=60)
    
    @staticmethod
    async def implement_caching():
        """
        Best Practice 3: Caching for Performance
        
        Cache frequently accessed resources and tool results.
        """
        from functools import lru_cache
        from hashlib import sha256
        
        class MCPCache:
            def __init__(self, ttl_seconds: int = 300):
                self.cache = {}
                self.ttl = ttl_seconds
            
            def _get_cache_key(self, tool_name: str, arguments: Dict[str, Any]) -> str:
                """Generate cache key from tool call"""
                key_data = f"{tool_name}:{json.dumps(arguments, sort_keys=True)}"
                return sha256(key_data.encode()).hexdigest()
            
            async def get_or_execute(self, tool_name: str, arguments: Dict[str, Any], executor):
                """Get from cache or execute tool"""
                cache_key = self._get_cache_key(tool_name, arguments)
                
                # Check cache
                if cache_key in self.cache:
                    cached_result, timestamp = self.cache[cache_key]
                    if time() - timestamp < self.ttl:
                        return cached_result
                
                # Execute and cache
                result = await executor(tool_name, arguments)
                self.cache[cache_key] = (result, time())
                return result
        
        return MCPCache()
    
    @staticmethod
    def implement_error_handling():
        """
        Best Practice 4: Comprehensive Error Handling
        
        Handle errors gracefully with proper logging.
        """
        import logging
        
        class MCPErrorHandler:
            def __init__(self):
                self.logger = logging.getLogger('mcp_server')
            
            async def safe_execute(self, func, *args, **kwargs):
                """Execute function with comprehensive error handling"""
                try:
                    return await func(*args, **kwargs)
                
                except ValueError as e:
                    self.logger.warning(f"Validation error: {e}")
                    return {
                        "error": {
                            "type": "validation_error",
                            "message": str(e)
                        }
                    }
                
                except TimeoutError as e:
                    self.logger.error(f"Timeout error: {e}")
                    return {
                        "error": {
                            "type": "timeout_error",
                            "message": "Operation timed out"
                        }
                    }
                
                except Exception as e:
                    self.logger.exception(f"Unexpected error: {e}")
                    return {
                        "error": {
                            "type": "internal_error",
                            "message": "An internal error occurred"
                        }
                    }
        
        return MCPErrorHandler()

# Demonstrate best practices
print("MCP Best Practices Summary:")
print("=" * 50)
print("\n1. **Security**")
print("   - Always validate and sanitize inputs")
print("   - Implement authentication and authorization")
print("   - Use timeouts for tool execution")
print("\n2. **Performance**")
print("   - Implement caching for expensive operations")
print("   - Use connection pooling for resources")
print("   - Consider async/streaming for large data")
print("\n3. **Reliability**")
print("   - Comprehensive error handling")
print("   - Graceful degradation")
print("   - Proper logging and monitoring")
print("\n4. **Scalability**")
print("   - Rate limiting to prevent abuse")
print("   - Horizontal scaling support")
print("   - Resource pooling")

### Integration Patterns

Here are common patterns for integrating MCP into your applications:

In [ ]:
# Pattern 1: Factory Pattern for MCP Tools
class MCPToolFactory:
    """
    Factory pattern for creating MCP tools dynamically.
    
    This pattern is useful when you need to:
    - Create tools based on configuration
    - Support multiple tool providers
    - Enable plugin architecture
    """
    
    def __init__(self):
        self.tool_creators = {}
    
    def register_tool_creator(self, tool_type: str, creator):
        """Register a tool creator function"""
        self.tool_creators[tool_type] = creator
    
    def create_tool(self, tool_config: Dict[str, Any]) -> MCPTool:
        """Create a tool based on configuration"""
        tool_type = tool_config.get("type")
        if tool_type not in self.tool_creators:
            raise ValueError(f"Unknown tool type: {tool_type}")
        
        return self.tool_creators[tool_type](tool_config)

# Pattern 2: Adapter Pattern for Different AI Frameworks
class MCPFrameworkAdapter:
    """
    Adapter pattern for integrating MCP with different AI frameworks.
    """
    
    def to_langchain(self, mcp_tool: MCPTool) -> Tool:
        """Convert MCP tool to LangChain tool"""
        pass
    
    def to_openai_function(self, mcp_tool: MCPTool) -> Dict[str, Any]:
        """Convert MCP tool to OpenAI function calling format"""
        return {
            "name": mcp_tool.name,
            "description": mcp_tool.description,
            "parameters": mcp_tool.input_schema
        }
    
    def to_anthropic_tool(self, mcp_tool: MCPTool) -> Dict[str, Any]:
        """Convert MCP tool to Anthropic tool format"""
        return {
            "name": mcp_tool.name,
            "description": mcp_tool.description,
            "input_schema": mcp_tool.input_schema
        }

# Pattern 3: Chain of Responsibility for Request Processing
class MCPRequestHandler:
    """
    Chain of responsibility pattern for processing MCP requests.
    """
    
    def __init__(self):
        self.next_handler = None
    
    def set_next(self, handler):
        self.next_handler = handler
        return handler
    
    async def handle(self, request: Dict[str, Any]) -> Optional[Dict[str, Any]]:
        if self.can_handle(request):
            return await self.process(request)
        elif self.next_handler:
            return await self.next_handler.handle(request)
        return None
    
    def can_handle(self, request: Dict[str, Any]) -> bool:
        raise NotImplementedError
    
    async def process(self, request: Dict[str, Any]) -> Dict[str, Any]:
        raise NotImplementedError

print("Common MCP Integration Patterns:")
print("\n1. Factory Pattern: Dynamic tool creation")
print("2. Adapter Pattern: Framework integration")
print("3. Chain of Responsibility: Request processing")
print("4. Observer Pattern: Event handling")
print("5. Strategy Pattern: Tool execution strategies")

## Summary and Next Steps

Congratulations! You've learned about the Model Context Protocol and how to integrate it with LangChain and LangGraph. Here's what we covered:

### Key Takeaways

1. **MCP Fundamentals**
   - Standardized protocol for AI-tool integration
   - Resources, tools, and prompts as core concepts
   - JSON-RPC 2.0 communication

2. **LangChain Integration**
   - Converting MCP tools to LangChain tools
   - Using MCP in agents and chains
   - Handling stateful interactions

3. **LangGraph Advanced Patterns**
   - Building complex workflows with MCP
   - State management across tool calls
   - Multi-phase processing

4. **Custom Server Development**
   - Implementing your own MCP servers
   - Protocol handling
   - Best practices for production

### Next Steps

1. **Explore Official MCP Servers**
   - File system server
   - Database connectors
   - API integrations

2. **Build Your Own Tools**
   - Start with simple tools
   - Add to existing applications
   - Share with the community

3. **Advanced Topics**
   - Streaming and real-time updates
   - Multi-server orchestration
   - Security and authentication

### Resources

- Official MCP Documentation: https://docs.anthropic.com/en/docs/mcp
- Python SDK: https://github.com/modelcontextprotocol/python-sdk
- LangChain MCP Adapters: https://github.com/langchain-ai/langchain-mcp-adapters
- Community Examples: https://github.com/modelcontextprotocol/examples

Happy building with MCP! 🚀